In [ ]:
#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("gabrielmanfredi/football-players-ratings")
#print("Path to dataset files:", path)

# Importation des packages et du Dataset

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

In [ ]:
from torch.autograd import Variable
import torch
torch.__version__

'2.5.1+cu124'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
df = pd.read_csv('data_football_ratings.csv')

In [ ]:
# grouper pour enlever la variance entre les noteurs

In [4]:
df_grouped = df.groupby(['player', 'match'], as_index=False).agg({
    'original_rating': 'mean',
    'competition': 'first',
    'team': 'first',
    'pos': 'first',
    'pos_role': 'first',
    'is_human': 'mean',
    'goals': 'first',
    'assists': 'first',
    'shots_ontarget': 'first',
    'shots_offtarget': 'first',
    'shotsblocked': 'first',
    'chances2score': 'first',
    'drib_success': 'first',
    'drib_unsuccess': 'first',
    'keypasses': 'first',
    'touches': 'first',
    'passes_acc': 'first',
    'passes_inacc': 'first',
    'crosses_acc': 'first',
    'crosses_inacc': 'first',
    'lballs_acc': 'first',
    'lballs_inacc': 'first',
    'grduels_w': 'first',
    'grduels_l': 'first',
    'aerials_w': 'first',
    'aerials_l': 'first',
    'poss_lost': 'first',
    'fouls': 'first',
    'wasfouled': 'first',
    'clearances': 'first',
    'stop_shots': 'first',
    'interceptions': 'first',
    'tackles': 'first',
    'dribbled_past': 'first',
    'tballs_acc': 'first',
    'tballs_inacc': 'first',
    'ycards': 'first',
    'rcards': 'first',
    'dangmistakes': 'first',
    'countattack': 'first',
    'offsides': 'first',
    'goals_ag_otb': 'first',
    'goals_ag_itb': 'first',
    'saves_itb': 'first',
    'saves_otb': 'first',
    'saved_pen': 'first',
    'missed_penalties': 'first',
    'owngoals': 'first',
    'degree_centrality': 'first',
    'betweenness_centrality': 'first',
    'closeness_centrality': 'first',
    'flow_centrality': 'first',
    'flow_success': 'first',
    'betweenness2goals': 'first',
    'win': 'first',
    'lost': 'first',
    'is_home_team': 'first',
    'minutesPlayed': 'first',
    'game_duration': 'first'
})

In [ ]:
df_grouped

,player,match,original_rating,competition,team,pos,pos_role,is_human,goals,assists,...,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,win,lost,is_home_team,minutesPlayed,game_duration
0,Aaron Cresswell,"Arsenal - West Ham, 4 - 1",5.840000,Premier League 2017-18,West Ham,DF,DC,0.500000,0,0,...,0.205287,0.653333,0.500000,0.919554,0.000000,0,1,0,90,90
1,Aaron Cresswell,"Bournemouth - West Ham, 3 - 3",6.215000,Premier League 2017-18,West Ham,DF,DC,0.500000,0,1,...,0.192622,0.687719,0.346667,0.919554,0.047619,0,0,0,90,90
2,Aaron Cresswell,"Brighton - West Ham, 3 - 1",6.640000,Premier League 2017-18,West Ham,MF,DML,0.500000,0,0,...,0.285065,0.804762,0.491525,1.001402,0.000000,0,1,0,90,90
3,Aaron Cresswell,"Burnley - West Ham, 1 - 1",6.120000,Premier League 2017-18,West Ham,DF,DL,0.500000,0,0,...,0.322586,0.670635,0.350877,0.000000,0.000000,0,0,0,90,90
4,Aaron Cresswell,"Chelsea - West Ham, 1 - 1",5.980000,Premier League 2017-18,West Ham,DF,DC,0.500000,0,0,...,0.093859,0.536508,0.283784,0.000000,0.000000,0,0,0,63,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21349,Zoltan Gera,"Hungary - Portugal, 3 - 3",6.060000,Euro 2016,Hungary,MF,DMC,0.500000,1,0,...,0.156701,0.603571,0.173077,0.000000,0.000000,0,0,0,46,90
21350,Zoltan Gera,"Iceland - Hungary, 1 - 1",6.130000,Euro 2016,Hungary,MF,MC,0.500000,0,0,...,0.148610,0.635338,0.514563,0.680416,0.000000,0,0,0,90,90
21351,Zoltan Stieber,"Austria - Hungary, 0 - 2",6.910000,Euro 2016,Hungary,Sub,Sub,0.000000,1,0,...,0.276523,0.548701,0.081081,0.166667,0.166667,1,0,0,11,90
21352,Zoltan Stieber,"Hungary - Portugal, 3 - 3",5.970000,Euro 2016,Hungary,Sub,Sub,0.500000,0,0,...,0.000000,0.402381,0.019231,0.000000,0.000000,0,0,0,90,90


In [5]:
#Création de dummies pour retirer les informations des chaines de caractères (compétition, équipe, poste, poste principal ?, évaluateur)
columns_to_dummies = ['competition', 'team', 'pos', 'pos_role']
df = pd.get_dummies(df_grouped, columns=columns_to_dummies, prefix=columns_to_dummies)

In [6]:
# Nous créons une colonne qui permet de vérifier le nombre d'occurence par joueur
# Cette variable permettra de rendre non sensible nos prédictions à la moyenne car si le joueur apparait une fois il a possibilité d'afficher des performances élevées par chance contrairement aux joueurs jouant plus souvent. Le nombre de fois où un joueur joue peut être signe de bonne perf.

df['nombre_apparitions'] = df['player'].map(df['player'].value_counts())

In [7]:
df = df.drop(columns=['match'])

In [34]:
numeric_df = df.select_dtypes(include=['number','bool'])
correlations = numeric_df.corr()['original_rating']
cols_to_keep = correlations[(correlations <= -0.1) | (correlations >= 0.1)].index.tolist()

if 'original_rating' not in cols_to_keep:
    cols_to_keep.append('original_rating')

df = df[cols_to_keep]

In [35]:
Y = df['original_rating']
X = df.drop(columns=['original_rating'])

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialiser le modèle Random Forest
rf = RandomForestRegressor(random_state=42)

# Grille d'hyperparamètres
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [ 10, 20, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [ 2, 4, 10],
    'max_features': ['auto', 'sqrt'],
}

# Configuration de la recherche sur grille avec validation croisée
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=3,
    n_jobs=-1
)

# Entraîner la recherche sur grille
grid_search.fit(X_train, y_train)

# Meilleur modèle
best_rf = grid_search.best_estimator_
print("Meilleurs hyperparamètres :", grid_search.best_params_)

# Prédictions et MSE
y_train_pred = best_rf.predict(X_train)
y_test_pred = best_rf.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print(f"MSE sur l'ensemble d'entraînement : {mse_train:.4f}")
print(f"MSE sur l'ensemble de test : {mse_test:.4f}")

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# ----- Gradient Boosting Regressor -----
gbr = GradientBoostingRegressor(random_state=42)
param_grid_gbr = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}
grid_search_gbr = GridSearchCV(gbr, param_grid_gbr, scoring='neg_mean_squared_error', cv=5, verbose=3, n_jobs=-1)
grid_search_gbr.fit(X_train, y_train)
best_gbr = grid_search_gbr.best_estimator_
print("Meilleurs hyperparamètres pour Gradient Boosting :", grid_search_gbr.best_params_)

# ----- XGBoost -----
xgbr = xgb.XGBRegressor(random_state=42)
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 10]
}
grid_search_xgb = GridSearchCV(xgbr, param_grid_xgb, scoring='neg_mean_squared_error', cv=5, verbose=3, n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)
best_xgb = grid_search_xgb.best_estimator_
print("Meilleurs hyperparamètres pour XGBoost :", grid_search_xgb.best_params_)

# ----- LightGBM -----
lgbr = lgb.LGBMRegressor(random_state=42)
param_grid_lgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 50, 100],
    'max_depth': [-1, 10, 20]
}
grid_search_lgb = GridSearchCV(lgbr, param_grid_lgb, scoring='neg_mean_squared_error', cv=5, verbose=3, n_jobs=-1)
grid_search_lgb.fit(X_train, y_train)
best_lgb = grid_search_lgb.best_estimator_
print("Meilleurs hyperparamètres pour LightGBM :", grid_search_lgb.best_params_)

# ----- CatBoost -----
catboost = CatBoostRegressor(random_state=42, silent=True)
param_grid_cat = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [3, 5, 10]
}
grid_search_cat = GridSearchCV(catboost, param_grid_cat, scoring='neg_mean_squared_error', cv=5, verbose=3, n_jobs=-1)
grid_search_cat.fit(X_train, y_train)
best_cat = grid_search_cat.best_estimator_
print("Meilleurs hyperparamètres pour CatBoost :", grid_search_cat.best_params_)

# ----- SVR -----
svr = SVR()
param_grid_svr = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale']
}
grid_search_svr = GridSearchCV(svr, param_grid_svr, scoring='neg_mean_squared_error', cv=5, verbose=3, n_jobs=-1)
grid_search_svr.fit(X_train, y_train)
best_svr = grid_search_svr.best_estimator_
print("Meilleurs hyperparamètres pour SVR :", grid_search_svr.best_params_)

# ----- Evaluation des modèles -----
models = {
    "Gradient Boosting": best_gbr,
    "XGBoost": best_xgb,
    "LightGBM": best_lgb,
    "CatBoost": best_cat,
    "SVR": best_svr
}

for name, model in models.items():
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    print(f"{name} - MSE Entraînement: {mse_train:.4f}, MSE Test: {mse_test:.4f}")

In [ ]:
# Initialiser un dictionnaire pour stocker les performances des modèles
model_performance = {
    "Model": [],
    "Dataset": [],
    "MSE": []
}

# Fonction pour calculer et ajouter les performances au dictionnaire
def add_model_performance(model_name, model, X_train, y_train, X_test, y_test):
    """Calcule et ajoute les MSE des ensembles d'entraînement et de test."""
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    model_performance["Model"].append(model_name)
    model_performance["Dataset"].append("Train")
    model_performance["MSE"].append(mse_train)
    model_performance["Model"].append(model_name)
    model_performance["Dataset"].append("Test")
    model_performance["MSE"].append(mse_test)

# Ajouter les performances des modèles
add_model_performance("Random Forest", best_rf, X_train, y_train, X_test, y_test)
add_model_performance("Gradient Boosting", best_gbr, X_train, y_train, X_test, y_test)
add_model_performance("XGBoost", best_xgb, X_train, y_train, X_test, y_test)
add_model_performance("LightGBM", best_lgb, X_train, y_train, X_test, y_test)
add_model_performance("CatBoost", best_cat, X_train, y_train, X_test, y_test)
add_model_performance("SVM", best_svr, X_train, y_train, X_test, y_test)

# Convertir les performances en DataFrame
performance_df = pd.DataFrame(model_performance)

# Définir les couleurs pour les ensembles Train et Test
colors = {"Train": "#1f77b4", "Test": "#ff7f0e"}

# Générer un graphique à barres groupées pour les MSE
plt.figure(figsize=(12, 6))
for dataset in ["Train", "Test"]:
    subset = performance_df[performance_df["Dataset"] == dataset]
    plt.bar(subset["Model"], subset["MSE"], alpha=0.8, color=colors[dataset], label=f"MSE - {dataset}")

# Ajouter des détails au graphique
plt.title("Performances des modèles (MSE)", fontsize=16)
plt.ylabel("MSE", fontsize=14)
plt.xlabel("Modèles", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, loc="upper right")
plt.tight_layout()
plt.show()

In [ ]:
# Normalisation des importances (échelle 0-1) par modèle
all_feature_importances["Normalized Importance"] = all_feature_importances.groupby("Model")["Importance"].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# Sélectionner les N caractéristiques les plus importantes pour chaque modèle
top_n = 10  # Modifier cette valeur pour afficher plus ou moins de caractéristiques
top_features = all_feature_importances.groupby("Model").apply(
    lambda x: x.nlargest(top_n, "Normalized Importance")
).reset_index(drop=True)

# Créer un graphique à barres groupées
plt.figure(figsize=(15, 8))
sns.barplot(
    data=top_features,
    x="Feature",
    y="Normalized Importance",
    hue="Model",
    dodge=True
)
plt.title("Importance normalisée des caractéristiques par modèle", fontsize=16)
plt.xticks(rotation=45, ha="right", fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Caractéristique", fontsize=14)
plt.ylabel("Importance normalisée", fontsize=14)
plt.legend(title="Modèle", fontsize=12)
plt.tight_layout()
plt.show()